In [1]:
import sys
import os
from typing import Optional
from tqdm import tqdm
sys.path.append(os.path.abspath('../'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import seaborn as sns
import os
from IPython.display import display, HTML

from utils import *
from utils.plot_utils import *

pd.set_option('display.max_rows', 500)

In [2]:
orig_df = "/n/holylabs/LABS/sitanc_lab/Users/mfli/critical-windows-lm/results/QANoiseDenoise/ReviseCW_model=meta-llama-Llama-3.1-8B-Instruct_dataset_name=competition_math_jump=0.50_decline=-0.30/df.csv"
revised_cw = "/n/holylabs/LABS/sitanc_lab/Users/mfli/critical-windows-lm/results/QANoiseDenoise/ReviseCW_model=meta-llama-Llama-3.1-8B-Instruct_dataset_name=competition_math_jump=0.50_decline=-0.30/dataset_with_cw_revised.csv"

## CoT over MATH dataset


In [3]:
combined_df = pd.read_csv(orig_df)
combined_df["dataset_source"]="MATH"
uniq_stop_fracs = [0]+list(map(lambda x:float(x), list(combined_df.stop_frac.unique())))

In [4]:
print(combined_df.shape)
print(len(combined_df.no_deno_string_ans.unique()))
combined_df = combined_df[~combined_df.formatted_answer.isna()]
print(combined_df.shape)

(296040, 23)
272477
(296040, 23)


In [5]:
len(combined_df.problem.unique())

4934

In [6]:
display(display_then_plot(combined_df, "is_right","orig_is_right","Fraction of noise-denoise is right"))
plt.close()

,0.05,0.1,0.3,0.5,0.7,0.9,orig_is_right
dataset_source,,,,,,,
MATH,0.475841,0.474078,0.469964,0.474787,0.476388,0.479145,0.475882


In [7]:
display(display_then_plot(combined_df, "stump_is_right","orig_is_right","Fraction of stump is right"))
plt.close()

,0.05,0.1,0.3,0.5,0.7,0.9,orig_is_right
dataset_source,,,,,,,
MATH,0.092015,0.116944,0.149777,0.170045,0.220105,0.375963,0.475882


In [8]:
display(display_then_plot(combined_df, "is_consistent",None,"Fraction of noise-denoise is consistent"))
plt.close()

stop_frac,0.05,0.10,0.30,0.50,0.70,0.90
dataset_source,,,,,,
MATH,0.446879,0.461066,0.518768,0.584901,0.666478,0.792542


In [9]:
display(display_then_plot(combined_df, "stump_is_consistent",None,"Fraction of stump is consistent"))
plt.close()

stop_frac,0.05,0.10,0.30,0.50,0.70,0.90
dataset_source,,,,,,
MATH,0.127888,0.150993,0.201257,0.236927,0.328131,0.586745


## See differences in accuracy for different critical windows

In [10]:
combined_df.groupby("problem")[["is_cw","orig_is_right"]].apply(lambda x:x.astype(float).mean()).reset_index().describe()

,is_cw,orig_is_right
count,4934.000000,4934.000000
mean,0.195176,0.475882
std,0.396377,0.499469
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,1.000000
max,1.000000,1.000000


In [11]:
df_by_problem = combined_df.groupby("problem")[["is_cw","orig_is_right"]].mean().reset_index()
df_by_problem.describe()

,is_cw,orig_is_right
count,4934.000000,4934.000000
mean,0.195176,0.475882
std,0.396377,0.499469
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,1.000000
max,1.000000,1.000000


In [12]:
# Compute means for orig_is_right based on is_cw
result1 = combined_df.groupby(["dataset_source", "is_cw"])["orig_is_right"].mean().unstack()
result1.columns = ['orig_is_right (is_cw=0)', 'orig_is_right (is_cw=1)']
display(result1)

# Compute means for is_cw based on orig_is_right
result2 = combined_df.groupby(["dataset_source", "orig_is_right"])["is_cw"].mean().unstack()
result2.columns = ['is_cw (orig_is_right=0)', 'is_cw (orig_is_right=1)']
display(result2)

# Function to compute p-values for orig_is_right based on is_cw
def compute_p_values_orig_is_right(group):
    group_is_cw_0 = group[group['is_cw'] == 0]['orig_is_right'].astype(float)
    group_is_cw_1 = group[group['is_cw'] == 1]['orig_is_right'].astype(float)
    _, p_value = ttest_ind(group_is_cw_0, group_is_cw_1, equal_var=False, nan_policy='omit')
    return p_value

# Function to compute p-values for is_cw based on orig_is_right
def compute_p_values_is_cw(group):
    group_orig_is_right_0 = group[group['orig_is_right'] == 0]['is_cw'].astype(float)
    group_orig_is_right_1 = group[group['orig_is_right'] == 1]['is_cw'].astype(float)
    _, p_value = ttest_ind(group_orig_is_right_0, group_orig_is_right_1, equal_var=False, nan_policy='omit')
    return p_value

# Group by dataset_source and compute p-values for both analyses
p_values_orig_is_right = combined_df.groupby('dataset_source').apply(compute_p_values_orig_is_right).reset_index()
p_values_orig_is_right.columns = ['dataset_source', 'p_value (orig_is_right by is_cw)']

p_values_is_cw = combined_df.groupby('dataset_source').apply(compute_p_values_is_cw).reset_index()
p_values_is_cw.columns = ['dataset_source', 'p_value (is_cw by orig_is_right)']

# Display the p-values
print(p_values_orig_is_right)
print(p_values_is_cw)

,orig_is_right (is_cw=0),orig_is_right (is_cw=1)
dataset_source,,
MATH,0.564845,0.109034


,is_cw (orig_is_right=0),is_cw (orig_is_right=1)
dataset_source,,
MATH,0.331787,0.044719


  dataset_source  p_value (orig_is_right by is_cw)
0           MATH                               0.0
  dataset_source  p_value (is_cw by orig_is_right)
0           MATH                               0.0


/tmp/ipykernel_2471026/3474905930.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  p_values_orig_is_right = combined_df.groupby('dataset_source').apply(compute_p_values_orig_is_right).reset_index()
/tmp/ipykernel_2471026/3474905930.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  p_values_is_cw = combined_df.groupby('dataset_source').apply(compute_p_values_is_cw).reset_index()


### Resampling before cw

In [13]:
## Resampling
resample = (combined_df.groupby("problem")["is_right_cw"].mean())
original = (combined_df.groupby("problem")["orig_is_right"].mean())
bernoulli_samples = np.random.binomial(n=1, p=resample)

from scipy.stats import ttest_ind
print((combined_df.groupby("problem")["orig_is_right"].mean()).mean())
print((combined_df.groupby("problem")["is_right_cw"].mean()).mean())
ttest_ind(bernoulli_samples, original.astype(float).values,   alternative="greater")

0.4758816376165383
0.4937373327928658


TtestResult(statistic=np.float64(1.7726541888725025), pvalue=np.float64(0.038158438599821806), df=np.float64(9866.0))

### Mode of cws

In [14]:
problem_to_mode = []
combined_df["is_right_cw_mode"]=combined_df["orig_is_right"]
for problem in tqdm(combined_df[combined_df.is_cw].problem.unique()):
    filtered_df = combined_df[combined_df.problem==problem]
    filtered_df = filtered_df[filtered_df.stop_frac==filtered_df.stop_frac_cw.unique()[0]]
    mode = filtered_df.no_deno_ans_format.mode()[0]
    is_right_mode = filtered_df[filtered_df.no_deno_ans_format==mode].is_right.iloc[0]
    problem_to_mode.append({"problem":problem,
                            "answer_mode":mode,
                            "is_right_sample":filtered_df.is_right.mean(),
                            "is_right_mode":is_right_mode,
                           })
    combined_df.loc[combined_df.problem==problem, "is_right_cw_mode"] = float(is_right_mode)
problem_to_mode_df = pd.DataFrame(problem_to_mode)

  0%|                                                                                                                    | 0/963 [00:00<?, ?it/s]/tmp/ipykernel_2471026/759230934.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  combined_df.loc[combined_df.problem==problem, "is_right_cw_mode"] = float(is_right_mode)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 963/963 [00:26<00:00, 36.57it/s]


In [15]:
problem_to_mode_df[["is_right_sample","is_right_mode"]].mean()

is_right_sample    0.200519
is_right_mode      0.275182
dtype: float64

In [16]:
mode = (combined_df.groupby("problem")["is_right_cw_mode"].mean())
original = (combined_df.groupby("problem")["orig_is_right"].mean())

from scipy.stats import ttest_ind
print(original.mean())
print(mode.mean())
ttest_ind(mode.astype(float).values, original.astype(float).values, alternative="greater")

0.4758816376165383
0.5083096878800162


TtestResult(statistic=np.float64(3.22310306146682), pvalue=np.float64(0.0006361104404211738), df=np.float64(9866.0))

## Applying it to both with and without critical windows

In [17]:
combined_df["regenerate_all"] = 0.05 

In [23]:
combined_df["regenerate_all_mode"]=np.nan
for problem in tqdm(combined_df.problem.unique()):
    filtered_df = combined_df[combined_df.problem==problem]
    filtered_df = filtered_df[filtered_df.stop_frac==filtered_df.regenerate_all.unique()[0]]
    mode = filtered_df.no_deno_ans_format.mode()[0]
    
    is_right_mode = filtered_df[filtered_df.no_deno_ans_format==mode].is_right.iloc[0]
    combined_df.loc[combined_df.problem==problem, "regenerate_all_mode"] = float(is_right_mode)
print(combined_df.regenerate_all_mode.value_counts())

regenerate_all_mode
1.0    174420
0.0    121620
Name: count, dtype: int64


In [24]:
mode_all = (combined_df.groupby("problem")["regenerate_all_mode"].mean())
original = (combined_df.groupby("problem")["orig_is_right"].mean())

from scipy.stats import ttest_ind
print(original.mean())
print(mode_all.mean())
ttest_ind(mode_all.values, original.astype(float).values, alternative="greater")

0.4758816376165383
0.5891771382245643


TtestResult(statistic=np.float64(11.350672547155037), pvalue=np.float64(5.619763104703847e-30), df=np.float64(9866.0))

In [25]:
display(combined_df.groupby(["problem","is_cw"])["orig_is_right"].mean().groupby("is_cw").mean())
display(combined_df.groupby(["problem","is_cw"])["is_right_cw_mode"].mean().groupby("is_cw").mean())
display(combined_df.groupby(["problem","is_cw"])["regenerate_all_mode"].mean().groupby("is_cw").mean())

is_cw
False    0.564845
True     0.109034
Name: orig_is_right, dtype: float64

is_cw
False    0.564845
True     0.275182
Name: is_right_cw_mode, dtype: object

is_cw
False    0.635860
True     0.396677
Name: regenerate_all_mode, dtype: float64

# Does asking the model to revise work?

In [26]:
cw_revised = pd.read_csv(revised_cw)

In [27]:
## Inspect data for a second
print(cw_revised["revise_string_ans"].iloc[0])
print()
print(cw_revised["revise_ans_format"].iloc[0])
print()
display(cw_revised.sample(10)[["revise_ans_string","revise_ans_format"]])

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Produce a correct solution to the following math question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Let $x_1,$ $x_2,$ $\dots,$ $x_{101}$ be positive real numbers such that $x_1^2 + x_2^2 + \dots + x_{101}^2 = 1.$  Find the maximum value of
\[x_1 x_2 + x_1 x_3 + \dots + x_1 x_{101}.\]


Think of the math question thoroughly step by step.     Please only respond with the answer after reasoning thoroughly. 

<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Let's think step by step:  We are given that $x_1^2 + x_2^2 + \dots + x_{101}^2 = 1.$  We want to find the maximum value of
\[x_1 x_2 + x_1 x_3 + \dots + x_1 x_{101}.\]
We can start by noting that the given sum can be rewritten as
\[x_1 \sum_{i = 2}^{101} x_i.\]
Since the sum of squares of all the numbers is equal to 1, we can think of each $x_i$ as the distance of the point from the origin in a 101-dimensional space. Then, the sum of squar

,revise_ans_string,revise_ans_format
386,The answer is $\boxed{-3}$.,-3
547,The answer is $\frac{7}{16}$.,\frac{7}{16}
591,The answer is $\boxed{-2}$.,-2
96,The answer is $\boxed{133}$.,133
693,The answer is $\boxed{\frac{13}{5}}$.,\frac{13}{5}
604,The answer is $4x-5$.,4x-5
191,The answer is $\boxed{4}$.,4
667,The answer is $\boxed{4}$.,4
333,The answer is $\frac{1}{2}$.,\frac{1}{2}
66,The answer is $\boxed{1}$.,1


In [28]:
orig_is_right = combined_df.groupby("problem")[["orig_is_right","is_cw"]].mean()

cw_and_orig = (
    cw_revised
    .merge(
        orig_is_right.reset_index(),
        on="problem",
        how="right",
        suffixes=("_revised", "_combined")
    )
    .assign(after_cw_is_right=lambda x: x["orig_is_right"])
)
cw_and_orig.loc[cw_and_orig["is_cw"]>=0.5,"after_cw_is_right"] = cw_and_orig.loc[cw_and_orig["is_cw"]>=0.5,"revise_is_right"] 

/tmp/ipykernel_2471026/1278281193.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[False False False False False False False False False False True False
 True True True False False False False False False False False False
 False True True False True False False False False False False False
 False False False False False False False True False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False True
 False False False False False False False True False False False False
 False False False True True True False True False True False False False
 False False False False True True False True True False False False False
 False False False True False True True False False False False False
 False False True True False False False True True False False False True
 False False True False False False 

In [29]:
## Original accuracy
print("orig acc: {:3f}".format(cw_and_orig.orig_is_right.mean()))

## Modified accuracty
print("revise accuracy: {:3f}".format(cw_and_orig.after_cw_is_right.mean()))


from scipy.stats import ttest_ind
ttest_ind(np.array(cw_and_orig.after_cw_is_right.values.astype(float)),
          np.array(cw_and_orig.orig_is_right.values),
         alternative="greater")

orig acc: 0.475882
revise accuracy: 0.500608


TtestResult(statistic=np.float64(2.457447089769284), pvalue=np.float64(0.007004960745416886), df=np.float64(9866.0))